In [ ]:
%matplotlib inline
import GEOparse
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import decomposition
from sklearn.feature_selection import f_classif
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import explained_variance_score, plot_roc_curve
import statsmodels.api as sm
from scipy.stats import ttest_ind
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklego.linear_model import LowessRegression

### Reading in from external source

In [ ]:
geo = "GSE40738"
data = GEOparse.get_GEO(geo=geo, destdir="../Dataset/Patnaik2017", silent=True)
#table = pd.read_csv("../Dataset/Fehlman2020/expression_matrix.csv", sep="\t")
#table = pd.read_csv("../Dataset/Wozniak2015/GSE64591_non-normalized.txt", sep="\t", header=5)

In [ ]:
table = pd.concat((val.table.iloc[:,1:] for val in data.gsms.values()), axis=1).transpose()
mirnas = list(data.gsms.values())[0].table.iloc[:,0]
gsm_list = np.array(list(data.gsms.values()))

In [ ]:
table

### Get controls

In [ ]:
set([list(data.gsms.values())[i].metadata["title"][0] for i in range(177)])

### Remove postop

In [ ]:
mask = np.array(["postop" not in k.metadata["title"][0] for k in gsm_list])
table = table.loc[mask]
gsm_list = gsm_list[mask]

### Seperate case and controls

In [ ]:
controls = np.array(["control" in k.metadata["title"][0] for k in gsm_list])
sick = 1 - controls

In [ ]:
sum(controls)

### Drop NAN

In [ ]:
table = table.dropna(1)

### T-test

In [ ]:
X = table

In [ ]:
X

In [ ]:
X_healthy, X_cancer = X[controls == 1], X[sick == 1]

In [ ]:
results = ttest_ind(X_healthy, X_cancer).pvalue

In [ ]:
lowest, pvalues = np.argsort(results)[:5], np.sort(results)[:5]

In [ ]:
mirnas.iloc[lowest], pvalues

### F-test

In [ ]:
t3 = table

In [ ]:
f, p = f_classif(t3, sick)

In [ ]:
np.mean(p)

### ANOVA

In [ ]:
t3

In [ ]:
X = np.array(sick).reshape(-1, 1)

In [ ]:
linreg = LinearRegression()
linreg.fit(X, t3)

In [ ]:
fitted = linreg.predict(X)
explained_variance_score(t3, fitted)

### Mean-variance-plot

In [ ]:
np.mean(table)

In [ ]:
def get_means_and_variances(table):
    return np.mean(table), np.var(table, ddof=1)

In [ ]:
def mean_variance_plot(table):
    means, variances = get_means_and_variances(table)
    plt.scatter(means, variances)
    plt.xlabel("mean")
    plt.ylabel("variance")
    plt.plot()

In [ ]:
mean_variance_plot(table)

### Log transformation

In [ ]:
log_table = table #np.log2(table)

In [ ]:
mean_variance_plot(log_table)

In [ ]:
norm_log_table = log_table - np.mean(log_table)

In [ ]:
norm_log_table /= np.sqrt(np.mean(np.var(norm_log_table, ddof=1)))

In [ ]:
mean_variance_plot(norm_log_table)

### PCA

In [ ]:
pca = decomposition.PCA(n_components=10)

In [ ]:
pca.fit(norm_log_table)

In [ ]:
pca.explained_variance_ratio_

In [ ]:
components = pca.transform(norm_log_table)

In [ ]:
components.shape

In [ ]:
sum(controls)

In [ ]:
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1)

ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)
targets = ['Cancer', 'Healthy']
colors = ['r', 'g']
ax.scatter(components[controls == False, 0], components[controls == False, 1], c = 'r', s = 10)
ax.scatter(components[controls == True, 0], components[controls == True, 1], c = 'g', s = 10)
ax.legend(targets)
ax.grid()

In [ ]:
components2 = components[components[:,0] < 20, :]
controls2 = controls[components[:,0] < 20]

In [ ]:
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1)

ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)
targets = ['Cancer', 'Healthy']
colors = ['r', 'g']
ax.scatter(components2[controls2 == False, 0], components2[controls2 == False, 1], c = 'r', s = 10)
ax.scatter(components2[controls2 == True, 0], components2[controls2 == True, 1], c = 'g', s = 10)
ax.legend(targets)
ax.grid()

### Linear regression adjustments

In [ ]:
gsm_list[30].metadata

In [ ]:
sex = np.array(["female" in k.metadata["characteristics_ch1"][2] if "gender" in k.metadata["characteristics_ch1"][2] else "female" in k.metadata["characteristics_ch1"][1] for k in gsm_list])
age = np.array([float(k.metadata["characteristics_ch1"][3].split(" ")[1].replace("NA", "NaN")) if "age" in k.metadata["characteristics_ch1"][3] else float(k.metadata["characteristics_ch1"][2].split(" ")[1].replace("NA", "NaN")) for k in gsm_list])

In [ ]:
covars = pd.DataFrame(np.array([sex, age]).transpose(), columns=["sex", "age"])

In [ ]:
covars = covars.fillna(covars.mean())

In [ ]:
linreg = LinearRegression()
linreg.fit(covars, norm_log_table)

In [ ]:
adj_norm_log_table = norm_log_table - linreg.predict(covars)

# PCA

In [ ]:
pca = decomposition.PCA(n_components=2)

In [ ]:
pca.fit(adj_norm_log_table)

In [ ]:
components = pca.transform(adj_norm_log_table)

In [ ]:
components

In [ ]:
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1)

ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)
targets = ['Cancer', 'Healthy']
colors = ['r', 'g']
ax.scatter(components[controls == False, 0], components[controls == False, 1], c = 'r', s = 10)
ax.scatter(components[controls == True, 0], components[controls == True, 1], c = 'g', s = 10)
ax.legend(targets)
ax.grid()

### Logistic Regression

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(adj_norm_log_table, controls, test_size=0.33, random_state=42)

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)

In [ ]:
plot_roc_curve(model, X_test, y_test)

### XGBoost

In [ ]:
model = XGBClassifier(use_label_encoder=False)
model.fit(X_train, y_train)

In [ ]:
plot_roc_curve(model, X_test, y_test)

### Export data

In [ ]:
lookup_table = data.gpls["GPL16016"].table

In [ ]:
lookup_table = lookup_table.set_index("ID")

In [ ]:
lookup_table.head()

In [ ]:
mirbase = lookup_table.loc[mirnas]["miRNA_ID_LIST"]

In [ ]:
from Scripts import converters

In [ ]:
mask = mirbase.notna().to_numpy()
mirbase = mirbase[mask]
adj_norm_log_table = adj_norm_log_table.loc[:, mask]

In [ ]:
mirbase = [m.split(",")[0] for m in mirbase]

In [ ]:
sequences = converters.canonical_to_seq(mirbase, True)

In [ ]:
sequences.count(None)

In [ ]:
sequences = np.array(sequences)
mask = sequences != None
sequences = sequences[mask]
adj_norm_log_table = adj_norm_log_table.loc[:, mask]
adj_norm_log_table /= adj_norm_log_table.var().mean()**0.5

In [ ]:
adj_norm_log_table.columns = sequences
adj_norm_log_table = adj_norm_log_table.loc[:,~adj_norm_log_table.columns.duplicated()]
adj_norm_log_table["cancer"] = sick
adj_norm_log_table.head()

In [ ]:
adj_norm_log_table.to_csv("../TransformedData/Patnaik2017_adjusted.csv", index=False)